In [1]:
# Importing libraries
import pandas as pd
from datetime import date
from time import sleep
from sqlalchemy import create_engine
import pandas.io.sql as sqlio
import psycopg2
import sys
sys.path.extend([
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava',
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava\Scripts'
])
from Functions import *
from StravaCredentials import *

conn = psycopg2.connect(
    host = hostname,
    dbname = database,
    user = username,
    password = pwd,
    port = port_id
)

In [2]:
# Getting Strava acccess token
access_token = GetToken(data)

# Getting an updated list of activities
# activities df - this does not include workout details
updated_workouts = retrieve_activities(access_token)

# Creating list to compare against not updated list
updated_workouts_list = list(updated_workouts['id'])

print(len(updated_workouts_list))

Getting page number: 1
Extraction of page 1 complete
Getting page number: 2
Extraction of page 2 complete
Getting page number: 3
Extraction of page 3 complete
Getting page number: 4
Extraction of page 4 complete
Getting page number: 5
Extraction of page 5 complete
Getting page number: 6
Extraction of page 6 complete
Extraction done
889


In [3]:
# Creating a cursor and also quering the database to get the current list of workouts from activity table
cur = conn.cursor()
query = "select * from activity"

# Turning query into dataframe - all activities
not_updated_workouts = sqlio.read_sql_query(query, conn)
print('Extracting data from SQL database.')

# closing connection
conn.close()

# Creating a list of ids from the list of activity ids
not_updated_workouts_list = list(not_updated_workouts['activity_id'])

Extracting data from SQL database.


C:\Users\Usuario\AppData\Local\Temp\ipykernel_4972\3562866985.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  not_updated_workouts = sqlio.read_sql_query(query, conn)


In [7]:
not_updated_workouts_list

[9927103494, 10823868460, 10856932883, 10817608022, 10840172152, 10850826461]

In [6]:
print('not updated workouts',len(not_updated_workouts_list))
print('updated workout',len(updated_workouts_list))

not updated workouts 6
updated workout 889


In [5]:
# How many new workouts will be added to database
print('Adding',len(updated_workouts_list) - len(not_updated_workouts_list),'workouts to Database.')

# Creating a list containing the missing workouts
# missing workouts = workouts to be added 
missing_workouts = list(set(updated_workouts_list).difference(not_updated_workouts_list))

Adding 883 workouts to Database.


In [4]:

# Calling function to extract missing workouts
missing_workouts_json = GetAllWorkouts(missing_workouts, access_token)

with open(missing_workouts_json, 'w') as json_file:
    json.dump(missing_workouts_json, json_file)

# Parsing workout json
missing_workouts_df = CleanWorkoutJson(missing_workouts_json)

# Parsing workout json to create laps table
laps_df = CleanLapsJson(missing_workouts_json)

# Creating the score columns from the workout json 
missing_workouts_df = CreateScoreColumns(missing_workouts_df)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2368\898452098.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  not_updated_workouts = sqlio.read_sql_query(query, conn)


Extracting data from SQL database.
Adding 883 workouts to Database.
Extracting all workouts, due to the API rate limit, this will take 02 : 12 hours or 132.45 minutes.
Extracting workout: 1
Extracting workout: 2
Extracting workout: 3
Extracting workout: 4
Extracting workout: 5
Extracting workout: 6
Extracting workout: 7
Extracting workout: 8
Extracting workout: 9
Extracting workout: 10
Extracting workout: 11


KeyboardInterrupt: 

In [ ]:
missing_workouts_df.to_csv(r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava\Outputs\missing_workouts_testing_update_script.csv')

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
from datetime import date
from time import sleep
from sqlalchemy import create_engine
import psycopg2
import sys
sys.path.extend([
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava',
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava\Scripts'
])
from Functions import *
from StravaCredentials import *

# Creating connection to database and creating engine to upload data
db_url = f'postgresql://{username}:{pwd}@{hostname}:{port_id}/{database}'
engine = create_engine(db_url)

# conn = psycopg2.connect(
#     host = hostname,
#     dbname = database,
#     user = username,
#     password = pwd,
#     port = port_id
# )


missing_workouts_df = pd.read_csv(r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava\Outputs\missing_workouts_testing_update_script.csv')

In [ ]:
# Dividing all_workouts_df to multiple tables to then upload to database
activity, activity_name, activity_coordinates, activity_details, activity_scores = DivideTables(missing_workouts_df)

In [ ]:
# Creating connection to database and creating engine to upload data
db_url = f'postgresql://{username}:{pwd}@{hostname}:{port_id}/{database}'
engine = create_engine(db_url)


# Sending data to Postgresql
print('Uploading data to database.')

activity.to_sql('activity', engine, if_exists='replace', index=False, method = 'multi')
activity_name.to_sql('activity_name', engine, if_exists='replace', index=False, method = 'multi')
activity_coordinates.to_sql('activity_coordinates', engine, if_exists='replace', index=False, method = 'multi')
activity_details.to_sql('activity_details', engine, if_exists='replace', index=False, method = 'multi')
activity_scores.to_sql('activity_scores', engine, if_exists='replace', index=False, method = 'multi')
laps_df.to_sql('laps', engine, if_exists='replace', index=False, method = 'multi')

print('Database updated.')

In [ ]:
# closing sql database connections
engine.dispose()
conn.close()
print('Connections closed.')